<a href="https://colab.research.google.com/github/dikkietrom/KITT-AI-Browser/blob/main/shap_e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
%cd ~/
!git clone https://github.com/openai/shap-e 


In [ ]:
%cd shap-e


In [9]:
!pip install PyTorch3D


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement PyTorch3D (from versions: none)
ERROR: No matching distribution found for PyTorch3D


In [ ]:
!pip install -e .


In [8]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

batch_size = 4
guidance_scale = 15.0
prompt = "a shark"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))

# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
          decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)



  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|████████████████████████████████████████| 890M/890M [00:08<00:00, 116MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

  0%|          | 0/64 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAAAAzAAAAMwAzMzMzMwBmMwAzZjMzZgBmZjNmZmZmZjNmmT…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIQAAAAAAAAzADMAMwAzMzMzMzMzZmYzZjNmZmZmZjOZZjNmmWZmmT…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAADMzAAAAMwAzMzMzMwAzZjMzZmYzZjNmZmZmZjNmmTOZmQ…

HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIMAAAAAAAAAMwAzMwBmMwAzZjMzZgBmZjNmZjNmmWaZmQAAAAAAAA…

/root/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/root/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(
